# HLMA 408: Moindres Carrés Ordinaires

***
> __Auteur__: Joseph Salmon
> <joseph.salmon@umontpellier.fr>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
from scipy.optimize import lsq_linear

# from sklearn import linear_model, preprocessing
from mpl_toolkits.mplot3d import Axes3D

import ipywidgets as widgets
from ipywidgets import interactive
from matplotlib.ticker import MaxNLocator

In [2]:
# Original url:
url = 'https://forge.scilab.org/index.php/p/rdataset/source/file/master/csv/datasets/cars.csv'
# Alternative url:
# url = 'http://josephsalmon.eu/enseignement/TELECOM/MDI720/datasets/cars.csv'
# path_target = "./cars.csv"
# download(url, path_target, replace=False)

In [3]:
%matplotlib widget

In [4]:
dat = pd.read_csv(url)
dat = dat.drop(columns='Unnamed: 0')
dat.columns = ['Vitesse (mph)', 'Distance (ft)']

# Beware dat['speed'].shape = (50,), issue with sklearn API need (50,1)
X = dat[['Vitesse (mph)']]
y = dat['Distance (ft)']

Xval = X.values.squeeze()


# Regression model (with sklearn)
# skl_linmod = linear_model.LinearRegression()
# skl_linmod.fit(X, y)
n_samples, _ = X.shape

matrix_constants=np.zeros((n_samples, 2))
matrix_constants[:, 0] = np.ones(n_samples)
matrix_constants[:, 1] = Xval
res_1 = lsq_linear(matrix_constants, y.values)


delta_x = Xval.max() - Xval.min()
delta_y = y.max() - y.min()

xmin_normal = Xval.min() - delta_x * 0.2
xmax_normal = Xval.max() + delta_x * 0.2
ymin_normal = y.min() - delta_y * 0.5
ymax_normal = y.max() + delta_y * 0.2

X_to_predict = np.linspace(xmin_normal, xmax_normal, num=50).reshape(50, 1)
X_to_predict = pd.DataFrame(X_to_predict, columns=['Vitesse (mph)'])



slopes = y / Xval
delta_slopes = slopes.max() - slopes.min()
n_grid_cplx = 50j
n_betas = int(n_grid_cplx.imag)
beta_0_grid, beta_1_grid = np.mgrid[ymin_normal:ymax_normal:n_grid_cplx,
                                    slopes.min() - 0.2 * delta_slopes:slopes.max() + 0.2 * delta_slopes:n_grid_cplx]

betas_1 = np.linspace(slopes.min() - 0.2 * delta_slopes,
                      slopes.max() + 0.2 * delta_slopes, n_betas)
betas_0 = np.linspace(ymin_normal - 0.2 * delta_y,
                      ymax_normal + 0.2 * delta_y, n_betas)

In [5]:
beta_mco_0 = res_1.x[0]
beta_mco_1 = res_1.x[1]

In [6]:
xlabels = dat.columns[0]
ylabels = dat.columns[1]

In [7]:
def funct_quad(beta_0, beta_1):
    """Quadratic function to be displayed."""
    # Compute: np.linalg.norm(y - Xval * beta_1 - beta_0)**2
    return np.linalg.norm(y)**2 + n_samples * beta_0**2 + np.linalg.norm(Xval)**2 * beta_1**2 - 2 * np.sum(y) * beta_0 - 2 * np.dot(y, Xval) * beta_1 + 2 * np.sum(Xval) * beta_0 * beta_1


Z = funct_quad(beta_0_grid, beta_1_grid)  # to speed up visualization
levels = MaxNLocator(nbins=30).tick_values(Z.min(), Z.max())
mappable = plt.cm.ScalarMappable(cmap=plt.cm.hot)
mappable.set_array(Z)


def plotting_level_set(ax, fig, beta_0, beta_1, Z):
    """Plotting level sets."""

    cs = ax.contourf(beta_0, beta_1, Z, alpha=.75,
                     cmap=plt.cm.hot, levels=levels)
    ax.plot(beta_mco_0, beta_mco_1, 'or', ms=12, label=r"Moindres carrés ordinaires")
    ax.contour(beta_0, beta_1, Z, colors='black',  levels=levels)
    cbar = fig.colorbar(cs)

In [8]:
# def show_MCO_3D(beta_0=-17, beta_1=4, azim=280):
beta_0 = -17
beta_1 = 4
azim = 280
y_by_line = beta_0 + beta_1 * X_to_predict

fig = plt.figure(
    figsize=(9, 9), num='Moindres carrés ordinaires (MCO) et optimisation')

fig.canvas.toolbar_visible = False

ax1 = fig.add_subplot(2, 2, 1)
ax1.set_xlim(left=xmin_normal, right=xmax_normal)
ax1.set_ylim(bottom=ymin_normal, top=ymax_normal)

ax1.plot(X, y, 'o', label=r'Données',
         markeredgecolor='k', markeredgewidth=1)
ax1.plot(X_to_predict, beta_mco_0 + X_to_predict * beta_mco_1, "--", color='red',
         linewidth=2, label=r"Moindres carrés ordinaires")
ax1_pred, = ax1.plot(X_to_predict, y_by_line,
                     linewidth=2, color='k', label=r"$x \to \beta_0 + \beta_1 x$")

ax1.legend(numpoints=1, loc=2)  # numpoints = 1 for nicer display
ax1.set_xlabel(xlabels)
ax1.set_ylabel(ylabels)
ax1.set_title(r"Données et moindres carrés"
              #                   + "\n" +
              #                   r"$\beta_0 = {0:.2f}, \beta_1 = \hat\beta_1 = {1:.2f}$".format(beta_0, beta_1)
              )

ax2 = fig.add_subplot(2, 2, 2, projection='3d')
ax2.scatter(beta_mco_0, beta_mco_1, 1.1 * np.linalg.norm(y - Xval * beta_mco_1 - beta_mco_0)
            ** 2, marker="o", edgecolors='k', s=130, color="red", alpha=1)
ax2_scatter_current = ax2.scatter(beta_0, beta_1, 1.1 * np.linalg.norm(y - Xval * beta_1 - beta_0)
                                  ** 2, marker="v", edgecolors='k', s=130, color="k", label=r"$\beta = (\beta_0,\beta_1)$", alpha=1)
ax2.set_title("Fonction à minimiser")
ax2.plot_surface(beta_0_grid, beta_1_grid, Z, cmap=mappable.cmap,
                 norm=mappable.norm, alpha=0.7, linewidth=0)
ax2.view_init(azim=azim, elev=20)
ax2.set_xlabel(r"Ordonnée à l'origine ($\beta_0$)")
ax2.set_ylabel(r"Pente ($\beta_1$)")

ax3 = fig.add_subplot(2, 2, 3)
plotting_level_set(ax3, fig, beta_0_grid, beta_1_grid, Z)
ax3_pts, = ax3.plot(beta_0, beta_1, 'kv', ms=12,
                    label=r"$x \to \beta_0 + \beta_1 x$")
ax3.set_title(r"Lignes de niveau de la fonction à minimiser" +
              "\n" + r'$(\beta_0,\beta_1) \to || y-X\beta_1 -\beta_0||^2$')
ax3.set_xlabel(r"Ordonnée à l'origine ($\beta_0$)")
ax3.set_ylabel(r"Pente ($\beta_1$)")
ax3.legend(loc=3)
fig.tight_layout(pad=0.25, h_pad=0.83, w_pad=0.3)
plt.show()


def update(beta_0=-17, beta_1=4, azim=280):
    y_by_line = beta_0 + beta_1 * X_to_predict
    ax1_pred.set_ydata(y_by_line)
    ax2_scatter_current._offsets3d = (
        [beta_0], [beta_1], [1.1 * np.linalg.norm(y - Xval * beta_1 - beta_0) ** 2])
    ax2.view_init(azim=azim, elev=20)
    ax3_pts.set_xdata(beta_0)
    ax3_pts.set_ydata(beta_1)
    fig.canvas.draw()


interactive_plot = interactive(update,
                               beta_0=widgets.FloatSlider(min=beta_0_grid.min(), max=beta_0_grid.max(),
                                                          value=1.1 * beta_mco_0, description=r'$\beta_0$'),
                               beta_1=widgets.FloatSlider(min=beta_1_grid.min(), max=beta_1_grid.max(),
                                                          value=1.1 * beta_mco_1, description=r'$\beta_1$'),
                               azim=widgets.IntSlider(min=0, max=360, step=1, value=280, description=r'Azimut'))
# output = interactive_plot.children[-1]
# output.layout.height = '500px'
interactive_plot

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=-19.337004379562046, description='$\\beta_0$', max=143.6, min=-57.0), …